In [2]:
# 05_FinalPolicy_OneDay_Kriging.py
# Single-day kriging map with halo highlight on the max station.
# - Uses HP paths
# - PyKrige >=1.7 fix (no 'nugget' kwarg; retry with tiny jitter)
# - Optional exclusion of flagged station(s) for kriging only

import os, glob, warnings, re
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
from fiona.env import Env
from shapely.ops import unary_union
from shapely.geometry import Point
from pykrige.ok import OrdinaryKriging
from numpy.linalg import LinAlgError
from scipy.spatial import cKDTree

# ---------- Paths (UPDATED to HP)
DATA_DIR  = r"C:\Users\HP\Documents\SpatialCARE\Daily\DailyGPKG"
PASIG_SHP = r"C:\Users\HP\Documents\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
OUT_DIR   = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\final"
os.makedirs(OUT_DIR, exist_ok=True)

# ---------- CRS, figure, grid, styling
WGS84  = "+proj=longlat +datum=WGS84 +no_defs"
UTM51  = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7,7), 150
GRID_RES_M = 100
VMIN, VMAX = 0, 100
CMAP = mpl.colormaps["RdBu_r"]  # keep your original palette
HALO_RADIUS_M, HALO_ALPHA_MAX, HALO_SIGMA_FRAC = 800, 0.35, 2.5
COVERAGE_MAX_DIST_M = 3000
VARIOGRAM_MODEL = "spherical"
RETRY_JITTER = 1e-3  # tiny noise (µg/m³) for retry if matrix is singular

# ---------- Exclusion (for kriging only) — optional
EXCLUDE_FOR_KRIGING = True
FLAG_MICRO_SITES = {
    "san antonio barangay hall",
    "san antonio brgy hall",
    "san antonio brgy. hall",
}
def _norm_name(s):
    return re.sub(r"\s+", " ", str(s).strip().lower())

# Optional coordinate-based buffer (set to None to disable)
BBQ_COORD_WGS84 = None   # e.g., (121.070000, 14.570000)
BBQ_RADIUS_M    = 25.0

# ---------- Select target date
target_date = "2025-02-06"   # <<< change this date
f = os.path.join(DATA_DIR, f"date_{target_date}.gpkg")
if not os.path.exists(f):
    raise SystemExit(f"Not found: {f}")

# ---------- Pasig boundary
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None:
    pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)
geom = unary_union(pasig_utm.geometry)

# ---------- Read the day’s data
g = gpd.read_file(f)
if g.crs is None:
    g = g.set_crs(WGS84)
if "geometry" not in g or g.geometry.is_empty.all():
    lon = next((c for c in g.columns if str(c).lower() in ("longitude","lon","x")), None)
    lat = next((c for c in g.columns if str(c).lower() in ("latitude","lat","y")), None)
    if lon and lat:
        g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
    else:
        raise SystemExit("No geometry or lon/lat columns to build geometry.")

pm_col = next((c for c in g.columns if str(c).lower() in ("pm25","pm_25","pm2_5","pm2.5")), None)
if pm_col is None:
    raise SystemExit("No PM2.5 column found.")

# ---------- Prepare points in UTM; de-duplicate exact XY; clip negatives
pts = g.to_crs(UTM51)[[pm_col,"geometry"]].copy()
pts["x"] = pts.geometry.x
pts["y"] = pts.geometry.y
pts = pts.groupby(["x","y"], as_index=False)[pm_col].mean()
pts = gpd.GeoDataFrame(pts, geometry=gpd.points_from_xy(pts["x"], pts["y"], crs=UTM51))

# ---------- Optional exclusion for kriging
if EXCLUDE_FOR_KRIGING:
    # Map station names (if available) to deduped points via spatial join
    st_col = next((c for c in ("stations","station","Station","STATION") if c in g.columns), None)
    if st_col is not None:
        g_tmp = g.to_crs(UTM51)[[st_col, "geometry"]].copy()
        g_tmp["station_norm"] = g_tmp[st_col].astype(str).map(_norm_name)
        pts = gpd.sjoin(pts, g_tmp[["station_norm","geometry"]], how="left", predicate="intersects")
        pts["is_flagged"] = pts["station_norm"].isin(FLAG_MICRO_SITES)
    else:
        pts["is_flagged"] = False

    # Coordinate buffer exclusion (if provided)
    if BBQ_COORD_WGS84 is not None:
        bbq_pt = gpd.GeoSeries([Point(BBQ_COORD_WGS84)], crs=WGS84).to_crs(UTM51).iloc[0]
        within_buf = pts.geometry.within(bbq_pt.buffer(BBQ_RADIUS_M))
        pts.loc[within_buf, "is_flagged"] = True

    pts_krig = pts[~pts["is_flagged"]].copy()
else:
    pts_krig = pts.copy()

if pts_krig.shape[0] < 3:
    raise SystemExit("Not enough points for kriging after exclusion (need >=3).")

# ---------- Coordinates & grid
x = pts_krig["x"].values
y = pts_krig["y"].values
z = pts_krig[pm_col].astype(float).clip(lower=0).values

minx, miny, maxx, maxy = geom.bounds
xs = np.arange(minx, maxx + GRID_RES_M, GRID_RES_M)
ys = np.arange(miny, maxy + GRID_RES_M, GRID_RES_M)
grid_x, grid_y = np.meshgrid(xs, ys)

# ---------- Kriging with retry using jitter (PyKrige >=1.7)
try:
    OK = OrdinaryKriging(x, y, z, variogram_model=VARIOGRAM_MODEL,
                         enable_plotting=False, verbose=False)
    zg, zv = OK.execute("grid", xs, ys, backend="loop")
    zg = np.array(zg)
except LinAlgError:
    z_jitter = z + np.random.normal(0, RETRY_JITTER, size=z.shape)
    OK = OrdinaryKriging(x, y, z_jitter, variogram_model=VARIOGRAM_MODEL,
                         enable_plotting=False, verbose=False)
    zg, zv = OK.execute("grid", xs, ys, backend="loop")
    zg = np.array(zg)

# ---------- Masks (coverage distance + within boundary)
tree = cKDTree(np.c_[x, y])
dists, _ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=1)
cover = (dists.reshape(grid_x.shape) <= COVERAGE_MAX_DIST_M)
centers = gpd.GeoSeries(gpd.points_from_xy(grid_x.ravel(), grid_y.ravel()), crs=UTM51)
inside  = centers.within(geom).values.reshape(zg.shape)
zmask = np.where(cover & inside, np.maximum(zg, 0.0), np.nan)

# ---------- Plot (halo on peak input point)
fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
im = ax.imshow(
    zmask, extent=[xs[0], xs[-1], ys[0], ys[-1]],
    origin="lower", cmap=CMAP, vmin=VMIN, vmax=VMAX
)
pasig_utm.boundary.plot(ax=ax, color="black", linewidth=0.8)

# Pick the max *input* point from pts_krig (consistent with the surface drivers)
i_max = int(np.nanargmax(z))
cx, cy = float(x[i_max]), float(y[i_max])

# Halo parameters
radius = HALO_RADIUS_M
sigma  = radius / HALO_SIGMA_FRAC
step   = max(20, GRID_RES_M // 2)
n      = int(2 * radius / step) + 1
hx = np.linspace(cx - radius, cx + radius, n)
hy = np.linspace(cy - radius, cy + radius, n)
HX, HY = np.meshgrid(hx, hy)
R2 = (HX - cx) ** 2 + (HY - cy) ** 2
alpha = np.exp(-R2 / (2 * sigma ** 2))
alpha[R2 > radius ** 2] = 0

halo = np.zeros((n, n, 4))
halo[..., 0] = 0.85  # R
halo[..., 1] = 0.15  # G
halo[..., 2] = 0.10  # B
halo[..., 3] = HALO_ALPHA_MAX * alpha  # A

ax.imshow(
    halo, extent=[cx - radius, cx + radius, cy - radius, cy + radius],
    origin="lower", interpolation="bilinear", zorder=3
)

ax.scatter([cx], [cy], marker="*", s=260, facecolor="white",
           edgecolor="black", linewidths=1.0, zorder=4)

# Find the max value at that input point for annotation
z_at_max = z[i_max]
ax.text(cx, cy, f"  max {z_at_max:.1f} µg/m³", fontsize=10, weight="bold", color="black")

# If you excluded flagged sites, optionally show them as X markers for transparency
if EXCLUDE_FOR_KRIGING and "is_flagged" in pts.columns and pts["is_flagged"].any():
    flagged = pts[pts["is_flagged"]]
    ax.scatter(flagged["x"], flagged["y"], marker="X", s=110, color="none",
               edgecolor="black", linewidths=1.1, zorder=5)

cb = plt.colorbar(im, ax=ax, shrink=0.8)
cb.set_label("Estimated PM₂.₅ (µg/m³)")
ax.set_title(f"Estimated PM₂.₅ • {target_date}")
ax.set_axis_off()

out = os.path.join(OUT_DIR, f"final_policy_{target_date}.png")
plt.tight_layout()
plt.savefig(out, bbox_inches="tight")
plt.close(fig)
print("Saved:", out)

Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\final\final_policy_2025-02-06.png


In [3]:
# 06_FinalPolicy_OneDay_Kriging_withRoads.py
# Single-day kriging map with halo highlight on the max station.
# Includes Pasig road network (primary, secondary, tertiary).
# No basemap.

import os, re
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from fiona.env import Env
from shapely.ops import unary_union
from shapely.geometry import Point
from pykrige.ok import OrdinaryKriging
from numpy.linalg import LinAlgError
from scipy.spatial import cKDTree

# ---------- Paths
DATA_DIR   = r"C:\Users\HP\Documents\SpatialCARE\Daily\DailyGPKG"
PASIG_SHP  = r"C:\Users\HP\Documents\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
ROADS_SHP  = r"C:\Users\HP\Documents\PhD Class\Shapefile\MM\Pasig\PasigRN.shp"
OUT_DIR    = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\final"
os.makedirs(OUT_DIR, exist_ok=True)

# ---------- CRS, figure, grid, styling
WGS84  = "+proj=longlat +datum=WGS84 +no_defs"
UTM51  = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7,7), 150
GRID_RES_M = 100
VMIN, VMAX = 0, 100
CMAP = mpl.colormaps["RdBu"]
HALO_RADIUS_M, HALO_ALPHA_MAX, HALO_SIGMA_FRAC = 800, 0.35, 2.5
COVERAGE_MAX_DIST_M = 3000
VARIOGRAM_MODEL = "spherical"
RETRY_JITTER = 1e-3  # tiny noise (µg/m³) for retry if singular

# ---------- Exclusion (for kriging only) — optional
EXCLUDE_FOR_KRIGING = True
FLAG_MICRO_SITES = {
    "san antonio barangay hall",
    "san antonio brgy hall",
    "san antonio brgy. hall",
}
def _norm_name(s):
    return re.sub(r"\s+", " ", str(s).strip().lower())

# Optional coordinate-based buffer (set to None to disable)
BBQ_COORD_WGS84 = None
BBQ_RADIUS_M    = 25.0

# ---------- Select target date
target_date = "2025-02-06"   # <<< change this date
f = os.path.join(DATA_DIR, f"date_{target_date}.gpkg")
if not os.path.exists(f):
    raise SystemExit(f"Not found: {f}")

# ---------- Pasig boundary
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None:
    pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)
geom = unary_union(pasig_utm.geometry)

# ---------- Roads
with Env(SHAPE_RESTORE_SHX="YES"):
    roads = gpd.read_file(ROADS_SHP)
if roads.crs is None:
    roads = roads.set_crs(WGS84)
roads_utm = roads.to_crs(UTM51)

# choose attribute column
col = None
for c in ("highway","fclass","type"):
    if c in roads_utm.columns:
        col = c
        break
if col is None:
    raise SystemExit("No road class column found.")

roads_utm[col] = roads_utm[col].astype(str).str.lower()
wanted = {"primary","secondary","tertiary"}
roads_sel = roads_utm[roads_utm[col].isin(wanted)].copy()

# ---------- Read the day’s data
g = gpd.read_file(f)
if g.crs is None:
    g = g.set_crs(WGS84)
if "geometry" not in g or g.geometry.is_empty.all():
    lon = next((c for c in g.columns if str(c).lower() in ("longitude","lon","x")), None)
    lat = next((c for c in g.columns if str(c).lower() in ("latitude","lat","y")), None)
    if lon and lat:
        g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
pm_col = next((c for c in g.columns if str(c).lower() in ("pm25","pm_25","pm2_5","pm2.5")), None)
if pm_col is None:
    raise SystemExit("No PM2.5 column found.")

pts = g.to_crs(UTM51)[[pm_col,"geometry"]].copy()
pts["x"]=pts.geometry.x; pts["y"]=pts.geometry.y
pts = pts.groupby(["x","y"], as_index=False)[pm_col].mean()
pts = gpd.GeoDataFrame(pts, geometry=gpd.points_from_xy(pts["x"], pts["y"], crs=UTM51))

# ---------- Exclusion
if EXCLUDE_FOR_KRIGING:
    st_col = next((c for c in ("stations","station","Station","STATION") if c in g.columns), None)
    if st_col is not None:
        g_tmp = g.to_crs(UTM51)[[st_col,"geometry"]].copy()
        g_tmp["station_norm"] = g_tmp[st_col].astype(str).map(_norm_name)
        pts = gpd.sjoin(pts, g_tmp[["station_norm","geometry"]], how="left", predicate="intersects")
        pts["is_flagged"] = pts["station_norm"].isin(FLAG_MICRO_SITES)
    else:
        pts["is_flagged"] = False
    if BBQ_COORD_WGS84 is not None:
        bbq_pt = gpd.GeoSeries([Point(BBQ_COORD_WGS84)], crs=WGS84).to_crs(UTM51).iloc[0]
        pts.loc[pts.geometry.within(bbq_pt.buffer(BBQ_RADIUS_M)), "is_flagged"] = True
    pts_krig = pts[~pts["is_flagged"]].copy()
else:
    pts_krig = pts.copy()

if pts_krig.shape[0] < 3:
    raise SystemExit("Not enough points for kriging after exclusion (need >=3).")

x = pts_krig["x"].values
y = pts_krig["y"].values
z = pts_krig[pm_col].astype(float).clip(lower=0).values

# ---------- Grid
minx,miny,maxx,maxy = geom.bounds
xs = np.arange(minx, maxx+GRID_RES_M, GRID_RES_M)
ys = np.arange(miny, maxy+GRID_RES_M, GRID_RES_M)
grid_x, grid_y = np.meshgrid(xs, ys)

# ---------- Kriging with jitter retry
try:
    OK = OrdinaryKriging(x,y,z, variogram_model=VARIOGRAM_MODEL,
                         enable_plotting=False, verbose=False)
    zg, zv = OK.execute("grid", xs, ys, backend="loop")
    zg = np.array(zg)
except LinAlgError:
    z_jitter = z + np.random.normal(0, RETRY_JITTER, size=z.shape)
    OK = OrdinaryKriging(x,y,z_jitter, variogram_model=VARIOGRAM_MODEL,
                         enable_plotting=False, verbose=False)
    zg, zv = OK.execute("grid", xs, ys, backend="loop")
    zg = np.array(zg)

# ---------- Masks
tree = cKDTree(np.c_[x,y]); dists,_ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=1)
cover = (dists.reshape(grid_x.shape) <= COVERAGE_MAX_DIST_M)
centers = gpd.GeoSeries(gpd.points_from_xy(grid_x.ravel(), grid_y.ravel()), crs=UTM51)
inside  = centers.within(geom).values.reshape(zg.shape)
zmask = np.where(cover & inside, np.maximum(zg,0.0), np.nan)

# ---------- Plot
fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
im = ax.imshow(zmask, extent=[xs[0], xs[-1], ys[0], ys[-1]],
               origin="lower", cmap=CMAP, vmin=VMIN, vmax=VMAX)
pasig_utm.boundary.plot(ax=ax, color="black", linewidth=0.8)

# plot road classes
style_map = {
    "primary":   {"linewidth": 2.0, "color": "#d62728"},
    "secondary": {"linewidth": 1.6, "color": "#1f77b4"},
    "tertiary":  {"linewidth": 1.2, "color": "#2ca02c"},
}
for klass, style in style_map.items():
    subset = roads_sel.loc[roads_sel[col] == klass]
    if not subset.empty:
        subset.plot(ax=ax, **style, zorder=4)

# halo at max input
i_max = int(np.nanargmax(z)); cx, cy = float(x[i_max]), float(y[i_max])
radius = HALO_RADIUS_M; sigma = radius / HALO_SIGMA_FRAC
step = max(20, GRID_RES_M//2); n = int(2*radius/step)+1
hx = np.linspace(cx-radius, cx+radius, n); hy = np.linspace(cy-radius, cy+radius, n)
HX, HY = np.meshgrid(hx, hy); R2 = (HX-cx)**2 + (HY-cy)**2
alpha = np.exp(-R2/(2*sigma**2)); alpha[R2>radius**2]=0
halo = np.zeros((n,n,4)); halo[...,0]=0.85; halo[...,1]=0.15; halo[...,2]=0.10; halo[...,3]=0.35*alpha
ax.imshow(halo, extent=[cx-radius, cx+radius, cy-radius, cy+radius],
          origin="lower", interpolation="bilinear", zorder=5)
ax.scatter([cx],[cy], marker="*", s=260, facecolor="white", edgecolor="black", linewidths=1.0, zorder=6)
ax.text(cx, cy, f"  max {z[i_max]:.1f} µg/m³", fontsize=10, weight="bold", color="black")

# flagged sites as X
if EXCLUDE_FOR_KRIGING and "is_flagged" in pts.columns and pts["is_flagged"].any():
    flagged = pts[pts["is_flagged"]]
    ax.scatter(flagged["x"], flagged["y"], marker="X", s=110, color="none",
               edgecolor="black", linewidths=1.1, zorder=7)

cb = plt.colorbar(im, ax=ax, shrink=0.8); cb.set_label("Estimated PM₂.₅ (µg/m³)")
ax.set_title(f"Estimated PM₂.₅ • {target_date}")
ax.set_axis_off()

out = os.path.join(OUT_DIR, f"final_policy_{target_date}.png")
plt.tight_layout(); plt.savefig(out, bbox_inches="tight"); plt.close(fig)
print("Saved:", out)


Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\final\final_policy_2025-02-06.png
